In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/

%cd Text_ming_project/

%ls

Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/Text_ming_project
amazon-2017.pdf  apple-2023.pdf                         meta-2021.pdf
amazon-2018.pdf  en_core_web_sm-3.8.0-py3-none-any.whl  meta-2022.pdf
amazon-2019.pdf  goog-10-k-2023.pdf                     meta-2023.pdf
amazon-2020.pdf  google-2017.pdf                        microsoft-2017.pdf
amazon-2021.pdf  google-2018.pdf                        microsoft-2018.pdf
amazon-2022.pdf  google-2019.pdf                        microsoft-2019.pdf
amazon-2023.pdf  google-2020.pdf                        microsoft-2020.pdf
apple-2017.pdf   google-2021.pdf                        microsoft-2021.pdf
apple-2018.pdf   google-2022.pdf                        microsoft-2022.pdf
apple-2019.pdf   meta-2017.pdf                          microsoft-2023.pdf
apple-2020.pdf   meta-2018.pdf                          requirements.txt
apple-2021.pdf   meta-2019.pdf                          Text_mining_project.ipynb
apple-2022.pdf   met

In [ ]:
%pip install pdfplumber PyPDF2
%pip install spacy
%pip wheel https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl
%pip install bertopic
!python3 -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 58.7 MB/s eta 0:00:00
  File was already downloaded /content/drive/MyDrive/Text_ming_project/en_core_web_sm-3.8.0-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 78.6 MB/s eta 0:00:00
✔ Download and installat

In [ ]:
import pdfplumber
import re
from PyPDF2 import PdfReader, PdfWriter
import matplotlib.pyplot as plt
import spacy

In [ ]:
def page_extract(pdf_path, chunk_size):
  with pdfplumber.open(pdf_path) as pdf:

    toc_page = None #finding the table of content page

    for _, page in enumerate(pdf.pages):
        text = page.extract_text()
        if re.search(r'\btable of contents|index\b', text.lower(), re.IGNORECASE):
            toc_page = text
            break

  section = "Mine Safety Disclosures" #the last subheading that we want to get all the information from the 10Q form

  #print(toc_page)
  # To capture the page number right after the header
  part_i_page = re.search(rf"{section}.*?(\d+)", toc_page)

  count = int(part_i_page.group(1))

  reader = PdfReader(pdf_path)
  pdf_writer = PdfWriter()

  content = []

  for i in range(count + 2):
      a = pdf_writer.add_page(reader.pages[i]).extract_text()
      content.append(a)


  full_text = " ".join(content)

  words = full_text.split()  # Split the text into words
  chunks = []
  chunk = []

  current_size = 0

  for word in words:
        word_length = len(word) + 1
        if current_size + word_length > chunk_size:
            chunks.append(" ".join(chunk))
            chunk = []
            current_size = 0
        chunk.append(word)
        current_size += word_length


  if chunk:
        chunks.append(" ".join(chunk))

  return chunks


nlp = spacy.load('en_core_web_sm')

def pre_process(content):
    preprocess = []
    for i in range(len(content)):
      text = re.sub(r'\d+', '', content[i])
      text = re.sub(r'-', ' ', text)
      text = re.sub(r'[^\w\s]', '', text)
      text = re.sub(r'\s+', ' ', text).strip()
      text = re.sub(r'_+', '', text)
      text = text.lower()

      doc = nlp(text)

      #lemmatized_text = [token.lemma_ for token in doc]

      final = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

      preprocess.append(" ".join(final))

    return preprocess


from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

def train_lda_model(documents, num_topics, passes=1):
    dictionary = Dictionary(documents)

    corpus = [dictionary.doc2bow(doc) for doc in documents]
    dictionary[0]
    model = LdaModel(corpus=corpus, id2word=dictionary.id2token, num_topics=num_topics, passes=passes)
    return model

2017

In [ ]:
meta_2017 = page_extract("meta-2017.pdf",chunk_size=800)
meta_2017_process = pre_process(meta_2017)

lda_content_2017 = [content.split() for content in meta_2017_process]
model_2017 = train_lda_model(lda_content_2017,num_topics=10,passes=10)
model_2017.print_topics()

[(0,
  '0.018*"result" + 0.018*"user" + 0.011*"financial" + 0.010*"product" + 0.010*"business" + 0.009*"continue" + 0.008*"addition" + 0.007*"account" + 0.007*"estimate" + 0.007*"affect"'),
 (1,
  '0.016*"law" + 0.013*"datum" + 0.013*"business" + 0.012*"subject" + 0.012*"tax" + 0.011*"protection" + 0.010*"regulation" + 0.010*"include" + 0.009*"service" + 0.009*"addition"'),
 (2,
  '0.022*"registrant" + 0.021*"act" + 0.020*"report" + 0.019*"exchange" + 0.017*"file" + 0.014*"check" + 0.014*"form" + 0.013*"k" + 0.013*"section" + 0.013*"mark"'),
 (3,
  '0.033*"user" + 0.029*"product" + 0.012*"result" + 0.009*"include" + 0.009*"action" + 0.009*"advertising" + 0.009*"change" + 0.008*"ad" + 0.008*"time" + 0.008*"business"'),
 (4,
  '0.012*"share" + 0.009*"stock" + 0.009*"business" + 0.009*"acquisition" + 0.009*"item" + 0.009*"price" + 0.008*"tax" + 0.008*"financial" + 0.008*"increase" + 0.007*"user"'),
 (5,
  '0.012*"statement" + 0.010*"continue" + 0.009*"cost" + 0.009*"program" + 0.008*"look

In [ ]:
from bertopic import BERTopic

topic_model_2017 = BERTopic()
topics, probs = topic_model_2017.fit_transform(meta_2017_process)

topic_model_2017.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,18,"[business, user, product, tax, datum, result, ..."
1,0,57,"[product, user, mobile, facebook, continue, ad..."
2,1,50,"[stock, common, class, tax, share, stockholder..."
3,2,43,"[law, business, protection, subject, regulatio..."
4,3,13,"[account, user, estimate, duplicate, false, nu..."


2018

In [ ]:
meta_2018 = page_extract("meta-2018.pdf",chunk_size=300)
meta_2018_process = pre_process(meta_2018)

lda_content_2018 = [content.split() for content in meta_2018_process]
model_2018 = train_lda_model(lda_content_2018,num_topics=10,passes=10)
model_2018.print_topics()

[(0,
  '0.039*"product" + 0.031*"user" + 0.016*"mobile" + 0.016*"datum" + 0.015*"facebook" + 0.014*"developer" + 0.011*"access" + 0.010*"party" + 0.009*"include" + 0.009*"service"'),
 (1,
  '0.017*"user" + 0.015*"account" + 0.012*"number" + 0.011*"datum" + 0.010*"repurchase" + 0.009*"business" + 0.008*"employee" + 0.008*"share" + 0.008*"duplicate" + 0.008*"include"'),
 (2,
  '0.029*"stock" + 0.028*"class" + 0.025*"common" + 0.017*"share" + 0.016*"business" + 0.016*"b" + 0.015*"tax" + 0.012*"change" + 0.009*"operation" + 0.009*"result"'),
 (3,
  '0.036*"user" + 0.015*"change" + 0.013*"product" + 0.012*"result" + 0.011*"stock" + 0.011*"continue" + 0.010*"engagement" + 0.009*"marketer" + 0.009*"price" + 0.008*"ability"'),
 (4,
  '0.017*"result" + 0.013*"time" + 0.013*"financial" + 0.012*"business" + 0.011*"patent" + 0.011*"right" + 0.010*"intellectual" + 0.009*"practice" + 0.009*"property" + 0.007*"party"'),
 (5,
  '0.016*"company" + 0.013*"account" + 0.009*"duplicate" + 0.009*"user" + 0.

In [ ]:
topic_model_2018 = BERTopic()
topics, probs = topic_model_2018.fit_transform(meta_2018_process)

topic_model_2018.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,78,"[stock, repurchase, program, statement, price,..."
1,0,145,"[product, user, mobile, ad, marketer, facebook..."
2,1,106,"[subject, business, result, law, regulation, a..."
3,2,46,"[datum, security, user, party, misuse, breach,..."
4,3,33,"[account, estimate, duplicate, user, false, ma..."
5,4,29,"[stock, class, common, voting, control, stockh..."
6,5,28,"[tax, rate, income, taxis, period, change, eff..."
7,6,19,"[file, registrant, exchange, report, filer, ac..."
8,7,18,"[patent, right, property, intellectual, propri..."


2019

In [ ]:
meta_2019 = page_extract("meta-2019.pdf",chunk_size=500)
meta_2019_process = pre_process(meta_2019)

lda_content_2019 = [content.split() for content in meta_2019_process]
model_2019 = train_lda_model(lda_content_2019,num_topics=10,passes=10)
model_2019.print_topics()

[(0,
  '0.031*"product" + 0.022*"user" + 0.022*"account" + 0.012*"metric" + 0.012*"family" + 0.012*"mobile" + 0.011*"people" + 0.011*"include" + 0.011*"use" + 0.011*"number"'),
 (1,
  '0.021*"datum" + 0.017*"user" + 0.015*"product" + 0.012*"certain" + 0.012*"service" + 0.009*"action" + 0.009*"law" + 0.008*"subject" + 0.008*"protection" + 0.008*"developer"'),
 (2,
  '0.019*"user" + 0.016*"product" + 0.014*"adversely" + 0.013*"affect" + 0.012*"business" + 0.011*"ability" + 0.010*"company" + 0.010*"result" + 0.008*"developer" + 0.007*"service"'),
 (3,
  '0.016*"tax" + 0.014*"court" + 0.011*"class" + 0.011*"district" + 0.007*"effective" + 0.007*"rate" + 0.007*"affect" + 0.007*"believe" + 0.006*"practice" + 0.006*"service"'),
 (4,
  '0.048*"product" + 0.013*"user" + 0.010*"hardware" + 0.009*"new" + 0.009*"example" + 0.008*"experience" + 0.008*"consumer" + 0.008*"information" + 0.007*"party" + 0.007*"significant"'),
 (5,
  '0.020*"account" + 0.015*"tax" + 0.012*"user" + 0.009*"business" + 0.

In [ ]:
topic_model_2019 = BERTopic()
topics, probs = topic_model_2019.fit_transform(meta_2019_process)

topic_model_2019.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,0,321,"[product, user, business, result, affect, incl..."
1,1,49,"[account, user, metric, estimate, family, prod..."


2020

In [ ]:
meta_2020 = page_extract("meta-2020.pdf",chunk_size=500)
meta_2020_process = pre_process(meta_2020)

lda_content_2020 = [content.split() for content in meta_2020_process]
model_2020 = train_lda_model(lda_content_2020,num_topics=10,passes=10)
model_2020.print_topics()

[(0,
  '0.057*"account" + 0.029*"estimate" + 0.028*"metric" + 0.021*"user" + 0.016*"family" + 0.016*"duplicate" + 0.015*"false" + 0.013*"violate" + 0.011*"number" + 0.009*"product"'),
 (1,
  '0.037*"product" + 0.034*"user" + 0.013*"action" + 0.012*"service" + 0.011*"advertising" + 0.010*"change" + 0.010*"include" + 0.009*"affect" + 0.009*"result" + 0.008*"ability"'),
 (2,
  '0.024*"stock" + 0.022*"common" + 0.022*"class" + 0.016*"business" + 0.015*"affect" + 0.014*"user" + 0.011*"adversely" + 0.011*"result" + 0.010*"control" + 0.010*"b"'),
 (3,
  '0.014*"business" + 0.014*"change" + 0.013*"tax" + 0.012*"product" + 0.011*"result" + 0.010*"include" + 0.009*"subject" + 0.008*"user" + 0.008*"require" + 0.007*"law"'),
 (4,
  '0.015*"product" + 0.011*"result" + 0.010*"operation" + 0.010*"require" + 0.009*"people" + 0.008*"company" + 0.007*"forward" + 0.007*"time" + 0.007*"common" + 0.007*"statement"'),
 (5,
  '0.014*"product" + 0.014*"business" + 0.011*"law" + 0.010*"subject" + 0.010*"includ

In [ ]:
topic_model_2020 = BERTopic()
topics, probs = topic_model_2020.fit_transform(meta_2020_process)

topic_model_2020.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,0,379,"[product, user, business, result, datum, inclu..."
1,1,47,"[account, user, metric, estimate, family, numb..."
2,2,16,"[tax, income, rate, taxis, change, period, eff..."


2021

In [ ]:
meta_2021 = page_extract("meta-2021.pdf",chunk_size=500)
meta_2021_process = pre_process(meta_2021)

lda_content_2021 = [content.split() for content in meta_2021_process]
model_2021 = train_lda_model(lda_content_2021,num_topics=10,passes=10)
model_2021.print_topics()

[(0,
  '0.015*"result" + 0.012*"product" + 0.010*"tax" + 0.010*"risk" + 0.009*"change" + 0.008*"financial" + 0.008*"business" + 0.008*"account" + 0.008*"report" + 0.007*"datum"'),
 (1,
  '0.023*"product" + 0.013*"change" + 0.012*"ability" + 0.012*"user" + 0.011*"business" + 0.011*"result" + 0.011*"include" + 0.011*"ad" + 0.010*"mobile" + 0.009*"service"'),
 (2,
  '0.031*"stock" + 0.027*"common" + 0.026*"class" + 0.013*"share" + 0.011*"b" + 0.011*"stockholder" + 0.008*"result" + 0.008*"future" + 0.007*"voting" + 0.007*"business"'),
 (3,
  '0.017*"people" + 0.013*"tax" + 0.013*"metric" + 0.012*"file" + 0.012*"account" + 0.011*"family" + 0.010*"user" + 0.010*"multiple" + 0.010*"complaint" + 0.009*"product"'),
 (4,
  '0.029*"product" + 0.018*"user" + 0.018*"business" + 0.016*"result" + 0.015*"continue" + 0.013*"service" + 0.011*"increase" + 0.011*"affect" + 0.010*"adversely" + 0.009*"growth"'),
 (5,
  '0.014*"law" + 0.012*"service" + 0.012*"product" + 0.011*"protection" + 0.010*"include" +

In [ ]:
topic_model_2021 = BERTopic()
topics, probs = topic_model_2021.fit_transform(meta_2021_process)

topic_model_2021.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,0,413,"[product, user, business, result, include, dat..."
1,1,50,"[account, user, metric, estimate, family, prod..."


2022

In [ ]:
meta_2022 = page_extract("meta-2022.pdf",chunk_size=300)
meta_2022_process = pre_process(meta_2022)

lda_content_2022 = [content.split() for content in meta_2022_process]
model_2022 = train_lda_model(lda_content_2022,num_topics=10,passes=10)
model_2022.print_topics()

[(0,
  '0.023*"business" + 0.017*"product" + 0.015*"content" + 0.015*"include" + 0.014*"affect" + 0.013*"result" + 0.011*"financial" + 0.011*"operation" + 0.011*"advertising" + 0.010*"protection"'),
 (1,
  '0.030*"tax" + 0.018*"time" + 0.017*"change" + 0.016*"user" + 0.014*"share" + 0.012*"period" + 0.012*"rate" + 0.011*"affect" + 0.011*"product" + 0.010*"base"'),
 (2,
  '0.021*"datum" + 0.016*"court" + 0.015*"action" + 0.012*"district" + 0.011*"product" + 0.011*"certain" + 0.010*"relate" + 0.009*"disclosure" + 0.009*"result" + 0.009*"consumer"'),
 (3,
  '0.050*"stock" + 0.043*"class" + 0.040*"common" + 0.019*"stockholder" + 0.017*"share" + 0.016*"b" + 0.013*"control" + 0.011*"voting" + 0.011*"director" + 0.010*"market"'),
 (4,
  '0.039*"product" + 0.024*"change" + 0.022*"ability" + 0.019*"user" + 0.015*"service" + 0.014*"result" + 0.013*"retain" + 0.011*"attract" + 0.011*"continue" + 0.011*"business"'),
 (5,
  '0.025*"account" + 0.016*"business" + 0.016*"subject" + 0.015*"change" + 0.

In [ ]:
topic_model_2022 = BERTopic()
topics, probs = topic_model_2022.fit_transform(meta_2022_process)

topic_model_2022.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,147,"[product, include, result, business, service, ..."
1,0,98,"[product, ad, user, mobile, engagement, market..."
2,1,86,"[account, metric, estimate, user, family, numb..."
3,2,63,"[datum, security, user, content, platform, bre..."
4,3,53,"[law, country, regulation, government, restric..."
5,4,49,"[court, district, file, complaint, action, dis..."
6,5,48,"[stock, common, class, stockholder, share, vot..."
7,6,42,"[result, financial, consumer, business, produc..."
8,7,42,"[reality, metaverse, virtual, technology, inve..."
9,8,40,"[business, practice, inquiry, subject, investi..."


2023

In [ ]:
meta_2023 = page_extract("meta-2023.pdf",chunk_size=300)
meta_2023_process = pre_process(meta_2023)

lda_content_2023 = [content.split() for content in meta_2023_process]
model_2023 = train_lda_model(lda_content_2023,num_topics=10,passes=10)
model_2023.print_topics()

[(0,
  '0.033*"product" + 0.030*"business" + 0.021*"affect" + 0.017*"user" + 0.017*"adversely" + 0.016*"mobile" + 0.016*"change" + 0.015*"system" + 0.014*"service" + 0.012*"retain"'),
 (1,
  '0.020*"product" + 0.015*"result" + 0.014*"affect" + 0.014*"time" + 0.012*"include" + 0.012*"user" + 0.012*"financial" + 0.012*"adversely" + 0.010*"revenue" + 0.010*"business"'),
 (2,
  '0.028*"product" + 0.020*"change" + 0.018*"user" + 0.012*"service" + 0.012*"european" + 0.011*"result" + 0.011*"implement" + 0.010*"use" + 0.010*"future" + 0.009*"datum"'),
 (3,
  '0.020*"include" + 0.019*"result" + 0.017*"tax" + 0.016*"business" + 0.015*"product" + 0.013*"significant" + 0.012*"operation" + 0.012*"financial" + 0.011*"law" + 0.010*"change"'),
 (4,
  '0.022*"result" + 0.011*"datum" + 0.010*"certain" + 0.010*"investment" + 0.010*"estimate" + 0.010*"business" + 0.008*"acquisition" + 0.008*"increase" + 0.007*"quarter" + 0.007*"addition"'),
 (5,
  '0.023*"product" + 0.016*"company" + 0.015*"new" + 0.015*"

In [ ]:
topic_model_2023 = BERTopic()
topics, probs = topic_model_2023.fit_transform(meta_2023_process)

topic_model_2023.get_topic_info().iloc[:, [0,1, 3]]

,Topic,Count,Representation
0,-1,151,"[product, business, result, affect, regulatory..."
1,0,97,"[law, regulation, certain, european, service, ..."
2,1,85,"[account, estimate, metric, user, family, dupl..."
3,2,61,"[court, district, file, motion, inc, dismiss, ..."
4,3,60,"[user, engagement, ad, marketer, product, time..."
5,4,58,"[stock, common, class, stockholder, share, div..."
6,5,54,"[datum, user, content, illicit, illegal, platf..."
7,6,53,"[metaverse, technology, ai, product, reality, ..."
8,7,51,"[european, united, privacy, states, datum, gdp..."
9,8,39,"[cybersecurity, risk, security, vulnerability,..."


trend across years

In [ ]:
import pandas as pd

data = {2017: meta_2017_process,2018:meta_2018_process, 2019: meta_2019_process, 2020: meta_2020_process, 2021: meta_2021_process, 2022: meta_2022_process, 2023: meta_2023_process}

def create_yearly_documents(data):
    documents = []

    for year, doc_list in data.items():
        for doc in doc_list:
            documents.append((doc, year))

    return documents

documents = create_yearly_documents(data)

# Convert the list of tuples into a DataFrame
df = pd.DataFrame(documents, columns=['text', 'year'])

print(df)

                                                   text  year
0     united statessecurities exchange commission wa...  2017
1     registrant telephone number include area code ...  2017
2     act duringthe precede month short period regis...  2017
3     knowledge definitive proxy information stateme...  2017
4     comply new revise financial accounting standar...  2017
...                                                 ...   ...
3717  november december total november announce boar...  2023
3718  common stock commence january expiration date ...  2023
3719  market condition investment opportunity share ...  2023
3720  ii item annual report form k additional inform...  2023
3721                                                     2023

[3722 rows x 2 columns]


In [ ]:
topic_model = BERTopic()

text = df['text'].tolist()

topics, probs = topic_model.fit_transform(text)

timestamps = df["year"].to_list()

topics_over_time = topic_model.topics_over_time(text, timestamps)

# Visualize topics over time
topic_model.visualize_topics_over_time(topics_over_time)

In [ ]:
topics_of_interest = [10,11,29,35,58,70]

topic_model.visualize_topics_over_time(topics_over_time, topics = topics_of_interest)